# A meta presentation
### Or how I stopped using clunkly tools like PowerPoint to present my work and learned how to use reveal.js with jupyter

In [264]:
# source: https://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Contents


1. What? Why?
2. Basic setup steps
2. Formatting
3. CSS to jazz things up
4. An example analysis - with code and charts
7. Publishing and running on your own machine
8. Using gitpages for hosting and sharing slides

# What?

1. reveal.js is a presentation framework to produce slides with HTML
2. Not only that you can use jupyter to build your slides!

# Why?

1. It is really nice for shareing code and analysis 

2. Also, as a training tool for coding

3. Get rid of all that tedious mucking about exporting images and playing around drawing boxes and whatever that make PowerPoint really dull 

4. They look really slick

5. INTERACTIVE BOKEH CHARTS! (This was the main reason I decided to use reveal.js)

6. PowerPoint is kind of boring in 2018 and all the cool data scientists will be sniggering behind your back if you use it

7. You can host slides remotely on the web (with gitpages) for easy access and sharing

# Basic setup steps

Select slideshow from the toolbar

![toolbar](files/images/menu "Toolbar")

This gives you a drop down at the top right of each cell with some options. These are...
![alt text](files/images/slide_type.png "options")

- Slide
    - Creates a new slide

- Fragment
    - create new part of a slide that appears when you press next like...

- ... this!

- Subside
    - allows you to scroll though vertically as well as horizontally like..

 - ...this

# Formatting text and images can be done using standard markdown

# A hash (#) preceding the text means it is formatted as heading 1

 ## two hashes means heading 2

### three means heading 3

#### and so on

# Lists

Adding a - before creates a...

- bullet
- point
- list

## While...

... adding a number and a full stop

1. creates a
2. numbered
3. list

## also...

```
**is bold text**
_italic text_
~~strikethrough~~

```
**bold text**

_italic text_ 

~~strikethrough~~ 


# More cool things with markdown

## Images

```
Format: ![Alt Text](location "hover text") 

```

e.g. 

```
![Goat](files/images/goat.png "mmmheaaaaa")
```

![Goat](files/images/goat.png "mmmheaaaaa")

## Links

```
[I'm an inline-style link with title](https://www.ons.gov.uk "ONS Homepage")
```
[I'm an inline-style link with title](https://www.ons.gov.uk "ONS Homepage")

Right, that is enough of that, onto the good stuff!

# Code 

One of the most useful things about using reveal.js with jupyter is that you can show code easily in your presentation.

## A clustering example

Adapted from: Machine learning with python cookbook, Chris Albon, O'Reilly media; http://shop.oreilly.com/product/0636920085423.do

In [88]:
#import modules and functions we need
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# load data
iris_data = datasets.load_iris().data
iris_labels = np.asarray(datasets.load_iris().target) #array for boolean later

print("iris dimensions: {}".format(iris_data.shape))

iris dimensions: (150, 4)


In [ ]:
#to make the colours match in the graphs
iris_labels = [2 if label == 1 else 1 if label == 2 else label for label in iris.target]

To plot this on a 2D scatter, we need to decompose the four features into two with Principle Components Analysis

In [24]:
iris_data_2D = PCA(n_components = 2).fit_transform(iris_data)

print("iris reduced dimensions: {}".format(iris_data_2D.shape))

iris reduced dimensions: (150, 2)


### Plot with bokeh

In [39]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, reset_output

#create figure
p = figure(title = "Principle components of the iris data",
           x_axis_label = "PCA component 1",
           y_axis_label = "PCA component 2",
           plot_width=600, 
           plot_height=400
          )

# plot points on the figure
p.circle(iris_data_2D[:,0], 
         iris_data_2D[:,1], 
         size=5, 
         color="navy", 
         alpha=0.5
        )

#plot inline in the notebook and show
output_notebook()
show(p)

Loading BokehJS ...

### K-means to seperate out the dataset

In [102]:
#scale
scaler = StandardScaler()
iris_data_scld = scaler.fit_transform(iris_data)

#create clusters
k_means_clst = KMeans(n_clusters = 3,
                      random_state = 42,
                      n_jobs = -1)

clusters = k_means_clst.fit(iris_data_scld)

### Lets have a look at the clusters

In [103]:
from bokeh.palettes import Category10
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap, linear_cmap

import numpy as np
import itertools as it

# create a convenience function

def plot_by_category(data, 
                     categories, 
                     palette = Category10,
                     plot_title = "K-means clusters of the iris data",
                     x_label = "PCA component 1",
                     y_label = "PCA component 2",
                     plot_dims = [600,400]
                    ):
    
    # create the figure
    p = figure(title = plot_title,
               x_axis_label = x_label,
               y_axis_label = y_label,
               plot_width = plot_dims[0], 
               plot_height = plot_dims[1]
              )

    # plotting by category is a little difficult as bokeh 0.13.0 is 
    # still under development

    #enables us to cycle through the colours in the palette 
    n_cat = np.max(categories) + 1
    colour_cycler = it.cycle(palette[n_cat])

    #plot each category in a differen colour    
    for cluster, colour in zip(np.unique(categories), colour_cycler):

        mask = categories == cluster
        
        p.circle(data[mask,0], 
                 data[mask,1], 
                 size=5, 
                 alpha=0.7,
                 color = colour
                )
    return p

p_clusters = plot_by_category(data = iris_data_2D,categories = clusters.labels_)
show(p_clusters)

### Has our clustering algorithm worked?

lets look at them in a tabbed plot

In [107]:
p_labels = plot_by_category(data = iris_data_2D, 
                            categories = iris_labels,
                            plot_title = "Labels of the iris data"
                           )

show(p_labels)

Looks a bit different... lets plot them in a more convenient way

In [108]:
from bokeh.models.widgets import Panel, Tabs

#create panels
panel_clusters = Panel(child = p_clusters, title = "Clusters")
panel_labels = Panel(child = p_labels, title = "Labels")

tabs = Tabs(tabs = [panel_clusters, panel_labels])

show(tabs)

or side by side

In [115]:
from bokeh.layouts import row

plot_row = row(children = [p_clusters, p_labels])

show(plot_row)

we can also plot on a single figure and jazz things up a bit 

In [229]:
from bokeh.models.sources import ColumnDataSource
from bokeh.models import CustomJS
from bokeh import events

def jazzy_plot(data, 
               clusters,
               labels,
               label_names,
               palette = Category10,
               plot_title = "K-means clusters of the iris data",
               x_label = "PCA component 1",
               y_label = "PCA component 2",
               plot_dims = [600,400]
              ):
    
    #tooltips for plot 
    t_tips= [("PC 1", "$x"),
                ("PC 2", "$y"),
                ("cluster", "@cluster"),
                ('label', '@label'),
                ('name', '@label_name')
               ]
    # create the figure
    p = figure(title = plot_title,
               x_axis_label = x_label,
               y_axis_label = y_label,
               plot_width = plot_dims[0], 
               plot_height = plot_dims[1],
               tooltips = t_tips
              )

    # plotting by category is a little difficult as bokeh 0.13.0 is 
    # still under development

    #enables us to cycle through the colours in the palette 
    n_cat = np.max(clusters) + 1
    colour_cycler = it.cycle(palette[n_cat])

    #plot each category in a differen colour     
    for cluster, colour, name in zip(np.unique(clusters), colour_cycler, np.unique(label_names)):
        
        #find where clusters and labels agree
        correct_mask = (clusters == cluster) & (labels == cluster) 
        incorrect_mask = (clusters == cluster) & (labels != cluster)
        
        # create a datasource 
        correct_src = ColumnDataSource({"PCA_1" : data[correct_mask,0],
                                        "PCA_2" : data[correct_mask,1],
                                        "cluster" : clusters[correct_mask],
                                        "label": labels[correct_mask],
                                        "label_name" : label_names[correct_mask]
                                       })
        
        incorrect_src = ColumnDataSource({"PCA_1" : data[incorrect_mask,0],
                                          "PCA_2" : data[incorrect_mask,1],
                                          "cluster" : clusters[incorrect_mask],
                                          "label": labels[incorrect_mask],
                                          "label_name" : label_names[incorrect_mask]
                                        })
   
        #plot circle if labels and clusters agree
        p.circle("PCA_1",
                 "PCA_2",
                 size=5, 
                 alpha=0.7,
                 color = colour,
                 source = correct_src,
                 legend = "{} correct".format(name)
                )
        
        #plot square if labels and cluster don't agree
        p.square("PCA_1",
                 "PCA_2",
                 size=5, 
                 alpha=0.7,
                 color = colour,
                 source = incorrect_src,
                 legend = "{} incorrect".format(name)
                )
    
    #set up the legend
    p.legend.location = "bottom_right"
    p.legend.orientation = "vertical"
    p.legend.click_policy = "hide"

    #make a event to get rid of the legend
    def show_hide_legend(legend = p.legend[0]):
        legend.visible = not legend.visible
    
    #turn into javascript and add to our plot
    p.js_on_event(events.DoubleTap, CustomJS.from_py_func(show_hide_legend))
        
        
    return p


### What does this look like?

In [199]:
# make some label names
names = {0 : "Setosa",
         1 : "Versicolour",
         2 : "Virginica"}

iris_names = np.asarray([names[label] for label in iris_labels]) # array for boolean

In [231]:
p_jazz = jazzy_plot(data = iris_data_2D,
                    clusters = clusters.labels_,
                    labels = iris_labels,
                    label_names = iris_names,
                    plot_title = "Clusters and targets comparison")

show(p_jazz)

### what if I want to hide all my code so no one sees?

**Use this handy button!**

In [256]:
# source: https://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

### But I did all my code somewhere else!

Well, that sounds quite sensible, you can just output the chart html...

In [238]:
import os
from bokeh.io import output_file
from bokeh.plotting import reset_output
p_jazz = jazzy_plot(data = iris_data_2D,
                    clusters = clusters.labels_,
                    labels = iris_labels,
                    label_names = iris_names,
                    plot_title = "Clusters and targets comparison")

reset_output() #make sure you call this in bokeh 0.13.0 as otherwise it will output everything you have created which can literally break your laptop
output_file("{}/figures/jazzy_plot.html".format(os.getcwd()))
show(p_jazz)

..then use an IFrame to display the HTML file

In [243]:
from IPython.display import IFrame

IFrame("figures/jazzy_plot.html".format(os.getcwd()), width = 800, height = 600)

# I've made some rad slides! 

neat

# But, how do I actually use them as slides?

Well there are a couple of methods....

...the first is to run a reveal.js server locally that you can view your slides.

In [ ]:
%%bash

jupyter nbconvert ~/Documents/Projects/revealjs-demo/meta-presentation.ipynb --to slides --post serve


(yes the %%bash magic command does mean you can run your slides from your notebook)

You can just generate the slides.html file and run it from there, if you have reveal.js cloned to your folder

You can add it to your folder with the below commands

In [262]:
%%bash
cd ~/Documents/Projects/revealjs-demo
git submodule add https://github.com/hakimel/reveal.js.git reveal.js

Cloning into 'revealjs-demo/reveal.js'...


In [ ]:
Then just drop the --post serve

In [265]:
%%bash

jupyter nbconvert ~/Documents/Projects/revealjs-demo/meta-presentation.ipynb --to slides

[NbConvertApp] Converting notebook /home/eddr/Documents/Projects/revealjs-demo/meta-presentation.ipynb to slides
[NbConvertApp] Writing 448648 bytes to /home/eddr/Documents/Projects/revealjs-demo/meta-presentation.slides.html


lets have a look...

In [266]:
%%bash
cd ~/Documents/Projects/revealjs-demo
ls

figures
images
menu
meta-presentation.ipynb
meta-presentation.slides.html
Pipfile
Pipfile.lock
reveal.js


# But I want to use it on another machine or share it with someone

The easiest way is to host it on github. You need to make sure you've added reveal.js as a submodule via git, like I did earlier then change your slides' name to index.html

In [269]:
%%bash

cp meta-presentation.slides.html index.html

In [270]:
%%bash
ls

figures
images
index.html
menu
meta-presentation.ipynb
meta-presentation.slides.html
Pipfile
Pipfile.lock
reveal.js


good now create a new (empty) repo on GitHub and intialise git 

In [271]:
%%bash
git init

Initialised empty Git repository in /home/eddr/Documents/Projects/revealjs-demo/.git/


then add and commit

In [272]:
%%bash
git add .
git commit -m "commiting my rad slides"

[master (root-commit) 07d1387] commiting my rad slides
 13 files changed, 32471 insertions(+)
 create mode 100644 .ipynb_checkpoints/meta-presentation-checkpoint.ipynb
 create mode 100644 Pipfile
 create mode 100644 Pipfile.lock
 create mode 100644 figures/jazzy_plot.html
 create mode 100644 images/goat
 create mode 100644 images/goat.png
 create mode 100644 images/menu
 create mode 100644 images/slide_type.png
 create mode 100644 index.html
 create mode 100644 menu
 create mode 100644 meta-presentation.ipynb
 create mode 100644 meta-presentation.slides.html
 create mode 160000 reveal.js


set the remote repo url

In [274]:
%%bash
git remote add origin https://github.com/ONSBigData/revealjs-demo.git

it is good to check this!

In [276]:
%%bash
git remote -v

origin	https://github.com/ONSBigData/revealjs-demo.git (fetch)
origin	https://github.com/ONSBigData/revealjs-demo.git (push)


and push... (you might need to do this in terminal to input your credentials

In [281]:
%%bash
git push origin master

fatal: could not read Username for 'https://github.com': No such device or address


Now go to your repo on github and select settings

![toolbar](files/images/settings.png "go(at) here!")

In [ ]:
Then scroll down to the gitpages section and tell it what branch to publish

In [ ]:
if this doesn't work , try this

In [288]:
%%bash

jupyter nbconvert ~/Documents/Projects/revealjs-demo/meta-presentation.ipynb --to slides 
cp meta-presentation.slides.html index.slides.html
git add .
git commit -m "commiting my rad slides"

[master 658d5c4] commiting my rad slides
 4 files changed, 32 insertions(+), 28778 deletions(-)
 delete mode 100644 index.html
 delete mode 100644 index.slides.html-


[NbConvertApp] Converting notebook /home/eddr/Documents/Projects/revealjs-demo/meta-presentation.ipynb to slides
[NbConvertApp] Writing 462383 bytes to /home/eddr/Documents/Projects/revealjs-demo/meta-presentation.slides.html


In [267]:
%%bash

jupyter nbconvert ~/Documents/Projects/revealjs-demo/meta-presentation.ipynb --to slides --post serve 
--SlidesExporter.reveal_theme=solarized
--SlidesExporter.reveal_scroll=True 
--SlidesExporter.reveal_transition=none

Process is terminated.
